# Submission Klasifikasi

**Dataset didapat dari proses machine learning clustering.**

link clustering : https://colab.research.google.com/drive/18Me8S2sHTURvVUswlcq2wuU0XueQMVvK?usp=sharing

## **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder


## **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [20]:
file_id = "1z6iomECewDR4BtnIa65lTmIlF9GyqQEb"
df = pd.read_csv(f"https://drive.google.com/uc?id={file_id}")
df

,Customer_Type,Product,Category,Unit_Price,Quantity,Discount,Total_Price,Region,Cluster
0,B2C,Coca-Cola,Soft Drinks,146.4330,892.0,0.000,1.041554e+05,Brandenburg,0
1,B2C,Club Mate,Soft Drinks,300.5158,1288.0,0.000,3.090988e+05,Niedersachsen,0
2,B2C,Mango Juice,Juices,439.9873,793.0,0.000,2.785888e+05,Niedersachsen,0
3,B2C,Pepsi,Soft Drinks,231.4442,694.0,0.000,1.281426e+05,Baden-Württemberg,0
4,B2C,Cranberry Juice,Juices,487.8061,1189.0,0.000,4.633324e+05,Baden-Württemberg,0
...,...,...,...,...,...,...,...,...,...
19994,B2C,Tomato Juice,Juices,429.3609,892.0,0.000,3.058374e+05,Bremen,0
19995,B2C,Volvic Touch,Water,183.6254,991.0,0.000,1.451862e+05,Bremen,0
19996,B2C,Cranberry Juice,Juices,459.9118,199.0,0.000,7.280369e+04,Bremen,0
19997,B2C,Beck's,Alcoholic Beverages,176.9839,397.0,0.000,5.597056e+04,Bremen,0


**Insight Data Loading :**

  Dataset disimpan dalam google drive, dan id file dalam google drive diambil. Setelah itu dataset dalam url google drive dibaca oleh pandas dengan format csv. File asli synthetic_beverage_sales_data.csv memiliki total baris lebih dari 8 juta, karena data yang terlalu banyak dapat menyebabkan overfitting, dan device saya tidak mencukupi terlalu banyak data. Maka dari itu, saya mengambil hanya 20.000 data ditengah dataset tersebut.

  **Informasi tambahan :**
  
  1. Banyak baris yang ada adalah 20.000
  2. Banyak kolom yang ada adalah 9

## **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [21]:
categorical_columns = ['Customer_Type', 'Category', 'Product', 'Region']

# Dictionary untuk menyimpan LabelEncoder
label_encoders = {}

# Encoding data
for column in categorical_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le  # Simpan encoder untuk inverse transform

# Tampilkan hasil encoding
print("Data setelah encoding:")
df.head()

Data setelah encoding:


,Customer_Type,Product,Category,Unit_Price,Quantity,Discount,Total_Price,Region,Cluster
0,1,6,2,146.4330,892.0,0.0,104155.4090,3,0
1,1,5,2,300.5158,1288.0,0.0,309098.8398,8,0
2,1,21,1,439.9873,793.0,0.0,278588.7808,8,0
3,1,28,2,231.4442,694.0,0.0,128142.6278,0,0
4,1,7,1,487.8061,1189.0,0.0,463332.4484,0,0


**Insight Label Encoding :**

  Kolom dengan tipe data kategorikal diubah ke dalam Label Encoding yang digunakan untuk mengubah data kategorikal menjadi bentuk numerik agar bisa digunakan dalam model machine learning.

  **Pada Dataset kali ini saya akan berikan label encode untuk kolom Customer_Type, Category, Product, dan Region.**

In [22]:
columns = df.columns
X = df[columns[:-1]] # kolom fitur
y = df['Cluster'] # kolom label

# Split data menjadi set pelatihan dan set uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tampilkan bentuk set pelatihan dan set uji untuk memastikan split
print(f"Training set shape: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Test set shape: X_test={X_test.shape}, y_test={y_test.shape}")

Training set shape: X_train=(15999, 8), y_train=(15999,)
Test set shape: X_test=(4000, 8), y_test=(4000,)


**Insight Data Splitting :**

  Dataset dipisah menjadi data train dan data test. Dimana data train diambil fitur nya dan dimasukkan kedalam x_train, lalu label nya dimasukkan kedalam y_train. Kemudian x_test diisi dengan fitur yang nantinya akan diuji oleh model machine learning. Dan y_test akan digunakan untuk evaluasi model.

  **Informasi tambahan :**
  
  1. Data latih 80% dari dataset = X_train=(15999, 8), y_train=(15999,)
  2. Data uji 20% dari dataset = X_test=(4000, 8), y_test=(4000,)

## **4. Membangun Model Klasifikasi**


### **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [23]:
# Buat model Decision Tree
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

# Prediksi
y_pred_dt = dt.predict(X_test)

# Evaluasi
print("Decision Tree Performance:")
print(classification_report(y_test, y_pred_dt))
print("Accuracy:", accuracy_score(y_test, y_pred_dt))


Decision Tree Performance:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3843
           1       1.00      1.00      1.00        31
           2       1.00      1.00      1.00       120
           3       1.00      1.00      1.00         6

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000

Accuracy: 1.0


In [24]:
# Buat model Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Prediksi
y_pred_rf = rf.predict(X_test)

# Evaluasi
print("\nRandom Forest Performance:")
print(classification_report(y_test, y_pred_rf))
print("Accuracy:", accuracy_score(y_test, y_pred_rf))



Random Forest Performance:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3843
           1       1.00      1.00      1.00        31
           2       1.00      1.00      1.00       120
           3       1.00      1.00      1.00         6

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000

Accuracy: 1.0


In [25]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Buat model KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

# Prediksi
y_pred_knn = knn.predict(X_test_scaled)

# Evaluasi
print("\nKNN Performance:")
print(classification_report(y_test, y_pred_knn))
print("Accuracy:", accuracy_score(y_test, y_pred_knn))



KNN Performance:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3843
           1       0.97      0.94      0.95        31
           2       0.96      0.93      0.94       120
           3       0.86      1.00      0.92         6

    accuracy                           1.00      4000
   macro avg       0.94      0.96      0.95      4000
weighted avg       1.00      1.00      1.00      4000

Accuracy: 0.99625


**Insight Model Selection**

  Saya menggunakan model
  1. K neighbours nearest (KNN)
  2. Decision Tree (DT)
  3. Random Forest (RF)

Dapat dilihat Decision Tree dan Random Forest memiliki nilai akurasi yang sama yaitu 1,00 artinya semua data dalam data uji diberikan label sesuai dengan data labelnya.
Sedangkan KNN memiliki akurasi 0.99625, artinya masih ada kekurangan dan algoritma KNN ini.

### **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

In [26]:
# Fungsi untuk mengevaluasi model dan menyimpan hasilnya dalam DataFrame
def evaluate_model(model_name, y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="weighted")
    recall = recall_score(y_true, y_pred, average="weighted")
    f1 = f1_score(y_true, y_pred, average="weighted")

    return [model_name, accuracy, precision, recall, f1]


In [27]:
# List untuk menyimpan hasil evaluasi
results = []
from sklearn.metrics import precision_score, recall_score, f1_score

# Decision Tree
results.append(evaluate_model("Decision Tree (DT)", y_test, y_pred_dt))

# Random Forest
results.append(evaluate_model("Random Forest (RF)", y_test, y_pred_rf))

# K-Nearest Neighbors
results.append(evaluate_model("K-Nearest Neighbors (KNN)", y_test, y_pred_knn))


In [28]:
# Konversi hasil evaluasi ke dalam DataFrame
df_results = pd.DataFrame(results, columns=["Model", "Accuracy", "Precision", "Recall", "F1-Score"])

# Atur jumlah angka desimal agar lebih rapi
df_results = df_results.round(4)

# Tampilkan hasil
print(df_results)


                       Model  Accuracy  Precision  Recall  F1-Score
0         Decision Tree (DT)    1.0000     1.0000  1.0000    1.0000
1         Random Forest (RF)    1.0000     1.0000  1.0000    1.0000
2  K-Nearest Neighbors (KNN)    0.9962     0.9962  0.9962    0.9962


**Insight Model Evaluation**

Dapat dilihat Decision Tree dan Random Forest memiliki nilai akurasi yang sama yaitu 1,00 artinya semua data dalam data uji diberikan label sesuai dengan data labelnya.
Sedangkan KNN memiliki akurasi 0.99625, artinya masih ada kekurangan dan algoritma KNN ini.

### **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [29]:
# Menggunakan GridSearchCV untuk mencari hyperparameter terbaik
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Menampilkan hyperparameter terbaik
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 100}


**Insight Tuning Model**

  Saya membutuhkan waktu yang cukup lama untuk Tuning Model dengan GridSearch method, disini saya menggunakan hypertuning terhadap model Random Forest.

### **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [35]:
# Menggunakan model terbaik
best_model = grid_search.best_estimator_
y_pred_tuned = best_model.predict(X_test)

# Menghitung ulang metrik evaluasi
accuracy = accuracy_score(y_test, y_pred_dt)
accuracy_tuned = accuracy_score(y_test, y_pred_tuned)
print("Tuned Accuracy:", accuracy_tuned)
print("Classification Report After Tuning:\n", classification_report(y_test, y_pred_tuned))

Tuned Accuracy: 1.0
Classification Report After Tuning:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      3843
           1       1.00      1.00      1.00        31
           2       1.00      1.00      1.00       120
           3       1.00      1.00      1.00         6

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000



In [36]:
# Konversi hasil evaluasi ke dalam DataFrame
df_results = pd.DataFrame(results, columns=["Model", "Accuracy", "Precision", "Recall", "F1-Score"])

# Atur jumlah angka desimal agar lebih rapi
df_results = df_results.round(4)

# Tampilkan hasil
print(df_results)

                       Model  Accuracy  Precision  Recall  F1-Score
0         Decision Tree (DT)    1.0000     1.0000  1.0000    1.0000
1         Random Forest (RF)    1.0000     1.0000  1.0000    1.0000
2  K-Nearest Neighbors (KNN)    0.9962     0.9962  0.9962    0.9962


### **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

In [37]:

# Bandingkan hasil sebelum dan sesudah tuning
print("Accuracy Improvement:", accuracy_tuned - accuracy)

# Identifikasi kelemahan model berdasarkan laporan klasifikasi
# Jika Precision atau Recall rendah, pertimbangkan untuk mengumpulkan data tambahan
# atau mencoba algoritma lain.

Accuracy Improvement: 0.0


##**Analisis Hasil Evaluasi Model Klasifikasi**
Berikut adalah rekomendasi tahapannya.

1. Akurasi model sebelum dan setelah dilakukan hyperparameter tuning dengan menggunakan metode GridSearch untuk model Random Forest tidak memiliki perubahan akurasi. Sebelum tuning Random Forest dapat menebak label sesuai dengan label data uji dengan akurasi 1.00 dan f1_score sempurna. Begitupun setelah tuning dilakukan.
2. Model ini mengalami overfitting disebabkan model belajar terlalu banyak dari data latih, termasuk detail dan noise yang tidak relevan. Overfitting dapat dilihat jika performa sangat baik pada data latih, tetapi buruk dalam data uji.
3. Penanganan overfitting dapat dilakukan dengan learning curve yang menunjukkan kesalahan rendah pada data latih, tetapi tinggi dalam data uji dapat menandakan overfitting. Kurva ini memvisualisasikan bahwa model terlalu kompleks.
4. Mengatasi nya juga bisa dilakukan :
- Cross-Validation
  
  Membagi data menjadi beberapa subset untuk memastikan model tidak overfitting pada subset tertentu dan dapat menggeneralisasi dengan baik.

- Early Stopping

  Menghentikan pelatihan saat model mulai overfit pada data validasi meskipun performa dalam data latih masih meningkat.

- Dan masih banyak lagi